<a href="https://colab.research.google.com/github/nam4chedspi/test/blob/master/opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# check GPU, memory, storage, TF
!nvidia-smi
!free -m
!df -h

import sys
print(sys.version)

import tensorflow as tf
print("Tensorflow version: "+tf.__version__)
if tf.test.gpu_device_name():
  print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
  print("Please install GPU version of TF")

import keras
print("Keras version: "+keras.__version__)

In [ ]:
!gdown --id 1nFMG-H2qEssUpP_RKGEz4tzFSKUy3xr0; unzip RockPaperScissors.zip -d /content/data; rm RockPaperScissors.zip
!gdown --id 1dhW-cddOyiqvHYHhvkLOJWskYdOM1e1O; unzip test.zip -d /content/test; rm test.zip

In [ ]:
# !curl -L "https://public.roboflow.com/ds/kJORCp6UNd?key=T8d3ssH991" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

data_dir = '/content/data'
img_width, img_height = 300, 300
batch_size = 256
seed=6

datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    # color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    seed=seed,
    subset='training') # set as training data
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    # color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    seed=seed,
    subset='validation') # set as validation data

print(validation_generator.class_indices)
LABELS = list(validation_generator.class_indices.keys())

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras import backend as K
from keras import optimizers
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
    'balanced',
    np.unique(train_generator.classes), 
    train_generator.classes)
class_weights_dict = dict(enumerate(class_weights))

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

opt = optimizers.RMSprop(learning_rate=0.001, epsilon=1e-07)
# opt = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# opt = optimizers.Adam(learning_rate=0.01)
epochs = 50

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    class_weight=class_weights_dict)
model.save('aaa.h5')

In [ ]:
from matplotlib import pyplot as plt

# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
 
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'],'r',linewidth=3.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

model_predict = load_model('aaa.h5')
model_predict.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

img = image.load_img('/content/test/paper/paper5_png.rf.7c669f01368a09cd3dad3ab605b1ee86.jpg', target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.

plt.imshow(x[0])                           
plt.axis('off')
plt.show()

img = image.load_img('/content/test/paper/paper5_png.rf.7c669f01368a09cd3dad3ab605b1ee86.jpg', target_size=(img_width, img_height), color_mode='grayscale')
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.
classes = model_predict.predict(x)
print(classes)
print(LABELS[np.argmax(classes)])